In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/malware-analysis-datasets-pe-section-headers/pe_section_headers.csv")
data.tail()

In [ ]:
howmanyna=data.isna().sum().sum()
print(howmanyna)

Doesn't have any na in this dataset.

I think hash value is not important. I think hash is used in signature base classification.

In [ ]:
x=data.iloc[:,1:5]
y=data.iloc[:,5:]

In [ ]:
%matplotlib inline
import seaborn as sns
sns.countplot(data=data,x="malware")

This data is unbalanced data. Classical method can't work with enough quality.

In [ ]:
sns.relplot(data=data,x="size_of_data",y="virtual_size",hue="malware")

İf Virtual Size is bigger than size of data İt's became malware. İt's my first hypothesis.

In [ ]:
sizedifference=data["virtual_size"]-data["size_of_data"]
sizedifference

In [ ]:
print(sizedifference.max())
print(sizedifference.min())

Size Difference is too big. 

Make classification with only 2 columns sizedifference and entrophy.

In [ ]:
newx=pd.DataFrame({"sizedif":sizedifference,"entr":data["entropy"]})
newx.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rfc=RandomForestClassifier()
#scores = cross_val_score(clf, iris.data, iris.target, cv=5)
cross_val_score(rfc,newx,y,cv=10)

Accuracy is not good metric for this 

In [ ]:
rfc=RandomForestClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,newx, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)

In [ ]:
conf_mat

To many false positive in this result. 

In [ ]:
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
rfc=DecisionTreeClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,newx, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

Performance is better but not enough.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scale_newx=sc.fit_transform(newx)

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel="rbf")
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(svm,scale_newx, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

RBF SVM have horrifically bad performance.

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel="linear")
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(svm,scale_newx, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

Now Use all the data and make classification

In [ ]:
rfc=RandomForestClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,x, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
rfc=DecisionTreeClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,x, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

Our Data Perform better than original data. 

PLot our data. 

In [ ]:
newplotx=pd.DataFrame({"sizedif":sizedifference,"entr":data["entropy"],"mal":y["malware"]})
newplotx.head()

In [ ]:
sns.relplot(data=newplotx,x="sizedif",y="entr",hue="mal")

I can't understand anything. These plots is can't say anything.

I forget Naive Bayes. Use Gaussian Naive Bayes for classification. 

In [ ]:
from sklearn.naive_bayes import GaussianNB
gbc=GaussianNB()
y_pred = cross_val_predict(gbc,newx, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

Can't work well. Nearly %50 %50 predict. 

These Approach have only we do under sampling.

In [ ]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(x, y)

In [ ]:
len(X_resamled)

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel="rbf")
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(svm,X_resampled, y_resampled, cv=10)
conf_mat = confusion_matrix(y_resampled, y_pred)
print(conf_mat)

This time Accuracy is crying.

In [ ]:
rfc=RandomForestClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,X_resampled, y_resampled, cv=10)
conf_mat = confusion_matrix(y_resampled, y_pred)
print(conf_mat)

Looks good. 

In [ ]:
rfc=DecisionTreeClassifier()
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(rfc,X_resampled, y_resampled, cv=10)
conf_mat = confusion_matrix(y_resampled, y_pred)
print(conf_mat)

Down Sampling is looking work. Lose some accuracy but more balanced decision making. 

In [ ]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X2_resampled, y2_resampled = cc.fit_resample(newx, y)

In [ ]:
rfc=RandomForestClassifier()
y_pred = cross_val_predict(rfc,X2_resampled, y2_resampled, cv=10)
conf_mat = confusion_matrix(y2_resampled, y_pred)
print(conf_mat)

In [ ]:
rfc=DecisionTreeClassifier()
y_pred = cross_val_predict(rfc,X2_resampled, y2_resampled, cv=10)
conf_mat = confusion_matrix(y2_resampled, y_pred)
print(conf_mat)

Our Data lost his virtie with downsampling. 

### Conclusion
This dataset is hard to classsify but have some observation and other things.

And lastly I miss GGPLOt :)